# KoBERT를 이용한 7가지 감정분류 모델

## Intro

In [121]:
from google.colab import drive

drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [122]:
cd /content/drive/MyDrive/SSAFY/자율/emotion/emotion_classification/

/content/drive/MyDrive/SSAFY/자율/emotion/emotion_classification


In [123]:
!pwd

/content/drive/MyDrive/SSAFY/자율/emotion/emotion_classification


In [124]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [125]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook, notebook

In [126]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [127]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [128]:
##GPU 사용 시
device = torch.device("cuda:0")

In [129]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


## Load Data

In [130]:
data1 = pd.read_csv('datasets/4차_14606.csv', encoding="CP949")
data2 = pd.read_csv('datasets/5차_10011.csv', encoding="CP949")
data = pd.concat([data1, data2])

data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,anger,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,anger,Sadness,1,Sadness,1,Sadness,1,Sadness,1,Sadness,1,27,male
4,5e27f90b5807b852d9e0157b,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad,Sadness,1,Sadness,1,Sadness,1,Sadness,2,Sadness,1,32,male


### Preprocessing Data

라벨이 여러개가 있어서 training하기 힘들기 때문에 라벨을 하나로 줄여준다.  
세기가 가장 큰 감정 하나만 남기는 작업을 수행한다.

In [131]:
d = {"Surprise" : 7, "Happiness": 6, "Disgust": 5, "Neutral": 4, "Fear": 3, "Angry": 2, "Sadness": 1}

for i in range(data.shape[0]):
  dic = {}
  for j in (3, 5, 7, 9, 11):
    dic[data.iloc[i, j]] = (dic.get(data.iloc[i, j], 0) + data.iloc[i, j + 1] + 1)
  m = 0
  e = ""
  for key, value in dic.items():
    if value > m:
      m = value
      e = key
    elif value == m:
      # e = "Neutral"
      e = max(e, key, key=lambda x: d[x])
  data.iloc[i, 2] = e

data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",Angry,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,Angry,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,Angry,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,Sadness,Sadness,1,Sadness,1,Sadness,1,Sadness,1,Sadness,1,27,male
4,5e27f90b5807b852d9e0157b,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,Sadness,Sadness,1,Sadness,1,Sadness,1,Sadness,2,Sadness,1,32,male


In [132]:
d = {"Surprise" : 7, "Happiness": 6, "Disgust": 5, "Neutral": 4, "Fear": 3, "Angry": 2, "Sadness": 1}

e = "Fear"
key = "Angry"
print(max(e, key, key=lambda x: d[x]))

Fear


`발화문`과 `상황`을 제외한 모든 컬럼을 삭제한다.

In [133]:
new_data = data.drop(columns=['wav_id', '1번 감정', '1번 감정세기', '2번 감정', '2번 감정세기', '3번 감정', '3번 감정세기', '4번 감정', '4번감정세기', '5번 감정', '5번 감정세기', '나이', '성별'])

new_data.head()

,발화문,상황
0,"어, 청소 니가 대신 해 줘!",Angry
1,둘 다 청소 하기 싫어. 귀찮아.,Angry
2,둘 다 하기 싫어서 화내.,Angry
3,그럼 방세는 어떡해.,Sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,Sadness


감정이 str이기 때문에 int형으로 매핑을 해준다.

In [134]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(new_data['상황'])
new_data['상황'] = encoder.transform(new_data['상황'])
new_data.head()

,발화문,상황
0,"어, 청소 니가 대신 해 줘!",0
1,둘 다 청소 하기 싫어. 귀찮아.,0
2,둘 다 하기 싫어서 화내.,0
3,그럼 방세는 어떡해.,5
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,5


In [135]:
new_data["상황"].value_counts()

5    13657
0     6284
2     1976
1     1372
4      786
3      440
6      102
Name: 상황, dtype: int64

In [136]:
uniform_data = pd.concat([new_data[new_data["상황"] == i][:440] for i in range(7)])

uniform_data.head()

,발화문,상황
0,"어, 청소 니가 대신 해 줘!",0
1,둘 다 청소 하기 싫어. 귀찮아.,0
2,둘 다 하기 싫어서 화내.,0
20,맨날 그래 얘는.,0
34,청소 좀 하고 살자.,0


In [137]:
uniform_data["상황"].value_counts()

5    440
3    440
1    440
4    440
2    440
0    440
6    102
Name: 상황, dtype: int64

In [138]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'Angry',
 1: 'Disgust',
 2: 'Fear',
 3: 'Happiness',
 4: 'Neutral',
 5: 'Sadness',
 6: 'Surprise'}

train과 test를 분리한다.

In [139]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(uniform_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 2193
test shape is: 549


In [140]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [141]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [158]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [159]:
train.head()

,발화문,상황
144,"밤새 기획안 작성했는데, 다 잘못됐대.",5
6301,그 방법도 되게 좋다. 한번 연락해봐야겠어.,3
43,응. 걷고 있어.,5
6983,어 고마워. 해볼게!,3
949,다행히 괜찮아.,4


In [160]:
data2.loc[738, '발화문']

'나 요즘 허리가 아픈 거 같아.\t'

In [161]:
data_train = BERTDataset(train.iloc, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test.iloc, 0, 1, tok, max_len, True, False)

In [162]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [163]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [164]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [165]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [166]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [167]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [168]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [169]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [170]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9476429224014282 train acc 0.125

epoch 1 train acc 0.514810924369748


epoch 2 batch id 1 loss 0.9964538216590881 train acc 0.71875

epoch 2 train acc 0.7604779411764706


epoch 3 batch id 1 loss 0.5554311275482178 train acc 0.859375

epoch 3 train acc 0.8379464285714285


epoch 4 batch id 1 loss 0.43141916394233704 train acc 0.890625

epoch 4 train acc 0.883140756302521


epoch 5 batch id 1 loss 0.25037088990211487 train acc 0.96875

epoch 5 train acc 0.9076943277310925


In [184]:
# 테스트 문장 예측
test_sentence = '정말 깜짝놀랐어'
test_label = 7 # 실제 질병

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['발화문', '상황']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor([[-0.3625,  0.2893,  0.0893, -0.1926, -1.0764, -0.8324,  2.6413]],
       device='cuda:0', grad_fn=<AddmmBackward>)



epoch 5 test acc 10.61890015015015

